# Topic Modeling

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit Learn
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import LsiModel
from gensim.models import CoherenceModel

# spaCy
import spacy

# nltk
from nltk.corpus import stopwords
from nltk.tokenize.regexp import RegexpTokenizer

## pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()


# Silence Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Load Text Libraries


### Load Corpora from spaCy and gensim

In [2]:
nlp = spacy.load('en_core_web_sm')

### Stop Words

In [3]:
def add_stop_words(word_list, list_stop_words):
    for word in word_list:
        list_stop_words.add(word)
        
def remove_stop_words(word_list, list_stop_words):
    for word in word_list:
        list_stop_words.remove(word)

#### Load Stop Words from NLTK and spaCy

In [4]:
stop_words_spacy = set(nlp.Defaults.stop_words)
stop_words_nltk = set(stopwords.words('english'))
full_stop_words = stop_words_spacy.union(stop_words_nltk)

#### Modify Stop Words Based on EDA

In [5]:
words_to_add = ['like', 'know', 'want', 'feel', 'going', 'think', 'reddit', 'imgur', 'pron', 'officially', 'story', 'month', 'week', 'time', 'day', 'year']

add_stop_words(words_to_add, full_stop_words)
remove_stop_words(['not'], full_stop_words)

## Read-In Data

In [23]:
health = pd.read_csv('../data/womens_health_preprocessed.csv', lineterminator='\n')
obsgyn = pd.read_csv('../data/fertility_and_pregnancy_preprocessed.csv')
pospar = pd.read_csv('../data/postpartum_preprocessed.csv')

## Modeling

### Model 1: LDA with Scikit Learn

#### Remove Digits from Text
After initial LDA models, it became clear that, while some numbers may occur frequently in posts, they do not add a lot of value to deriving meaning from the topics. Therefore, digits will be removed from the text.

In [24]:
def remove_digits(df, column1, column2):
    df[column1] = df[column1].str.replace('[0-9]+', '', regex = True)
    df[column2] = df[column2].str.replace('[0-9]+', '', regex = True)
    
# https://stackoverflow.com/questions/47010044/how-to-remove-numeric-characters-present-in-countvectorizer

dfs = [health, obsgyn, pospar]

for df in dfs:
    remove_digits(df, 'total_text', 'lemma_tokens')

In [9]:
# def remove_phrase(df, column):
#     df[column] = df[column].str.replace([deleted]', '')

# remove_phrase(obsgyn, 'lemma_tokens')

#### Function to Build LDA Model

In [10]:
def build_lda(df, column, n_topics, stop_words = full_stop_words):
    tvec = TfidfVectorizer(max_df = 0.98, min_df = 0.005, ngram_range = (1,2), stop_words = stop_words)
    dtm = tvec.fit_transform(df[column])
    
    LDA = LatentDirichletAllocation(n_components = n_topics, random_state = 42, n_jobs = -1)
    LDA.fit(dtm)
    
    vocab = tvec.get_feature_names()
    topics = LDA.components_
    
    print(f'The number of topics is {n_topics}.')
    print(f'Log-Likelihood Score: {round(LDA.score(dtm), 3)}')
    print(f'Perplexity Score: {round(LDA.perplexity(dtm), 3)}')
    print('\n')
    
    for index, topic in enumerate(topics):
        print(f'The top 15 words for topic {index}')
        print([vocab[index] for index in topic.argsort()[-15:]])
        print('\n')
        
    pyLDAvis.sklearn.prepare(LDA, dtm, tvec)
    
    return LDA, dtm, tvec

In [11]:
def grid_search_sk_lda(df, column, n_topics, step_size, stop_words = full_stop_words):
    for i in range(0, n_topics+1, step_size):
        build_lda(df, column, n_topics, stop_words = full_stop_words)
        print('- - - - - - - - - - - - - - - - - -')

#### Grid Search Num Topics for General Health Data

In [12]:
model, dtm, vectorizer = build_lda(health, 'lemma_tokens', 17)

The number of topics is 17.
Log-Likelihood Score: -1144294.513
Perplexity Score: 2371.087


The top 15 words for topic 0
['view', 'hormone', 'surgery', 'woman', 'pcos', 'uterus', 'cm', 'estrogen', 'menstrual cycle', 'progesterone', 'cycle', 'fibroid', 'tube', 'menstrual', 'menopause']


The top 15 words for topic 1
['experience', 'surgery', 'sharp', 'right', 'doctor', 'pelvic', 'left', 'ultrasound', 'ovarian cyst', 'breast', 'ovary', 'ovarian', 'delete', 'cyst', 'pain']


The top 15 words for topic 2
['painful', 'skin', 'look', 'small', 'notice', 'lump', 'bra', 'boob', 'nipple', 'bump', 'hurt', 'not', 'labia', 'breast', 'vagina']


The top 15 words for topic 3
['travel', 'ex', 'home', 'bag', 'deal', 'love', 'work', 'ring', 'help', 'favorite', 'guy', 'sleep', 'app', 'date', 'tip']


The top 15 words for topic 4
['pound', 'weight gain', 'not', 'fat', 'gain weight', 'food', 'lose weight', 'exercise', 'healthy', 'diet', 'lose', 'body', 'gain', 'eat', 'weight']


The top 15 words for topic 

In [13]:
pyLDAvis.sklearn.prepare(model, dtm, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12    -0.142264  0.000083       1        1  17.140148
7      0.019594 -0.049051       2        1   8.768376
15     0.151959 -0.044398       3        1   7.626300
11    -0.139204 -0.018248       4        1   7.089228
8     -0.091693  0.027450       5        1   6.837143
6     -0.189010  0.000850       6        1   6.675170
2     -0.034792 -0.022233       7        1   6.325626
14     0.046715  0.048231       8        1   5.955333
9      0.117891 -0.077248       9        1   5.799297
10     0.058838 -0.024691      10        1   5.652778
13     0.128531 -0.043675      11        1   5.399754
1      0.082039  0.078204      12        1   4.179797
4     -0.099813  0.032631      13        1   3.662595
5      0.148929 -0.098675      14        1   2.788145
16     0.027038 -0.078600      15        1   2.723929
3     -0.181363 -0.031094      16        1   2.063970
0      0.096605  0.300462      17        1   1.312410, topic_info=               Term        Freq       Total Category  logprob  loglift
565            hair  699.000000  699.000000  Default  30.0000  30.0000
137   birth control  551.000000  551.000000  Default  29.0000  29.0000
136           birth  568.000000  568.000000  Default  28.0000  28.0000
279         control  580.000000  580.000000  Default  27.0000  27.0000
1227            sex  718.000000  718.000000  Default  26.0000  26.0000
...             ...         ...         ...      ...      ...      ...
1158         remove   24.580714  183.120056  Topic17  -4.3646   2.3251
1570          woman   39.829217  420.585734  Topic17  -3.8820   1.9763
35              age   18.794893  157.518443  Topic17  -4.6330   2.2073
778             low   15.036542  232.728274  Topic17  -4.8561   1.5939
1393        symptom   13.083111  359.175268  Topic17  -4.9953   1.0208

[861 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         3  0.072178  abdoman
0         9  0.048119  abdoman
0        10  0.264652  abdoman
0        11  0.024059  abdoman
0        12  0.553364  abdoman
...     ...       ...      ...
1588      4  0.056172   yellow
1588      6  0.037448   yellow
1588      7  0.093620   yellow
1588      9  0.037448   yellow
1588     10  0.056172   yellow

[5718 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 8, 16, 12, 9, 7, 3, 15, 10, 11, 14, 2, 5, 6, 17, 4, 1])

#### Grid Search Num Topics for Fertility and Pregnancy

In [25]:
model, dtm, vectorizer = build_lda(obsgyn, 'lemma_tokens', 30)

KeyboardInterrupt: 

In [15]:
pyLDAvis.sklearn.prepare(model, dtm, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
11    -0.062886 -0.032653       1        1  8.323176
13    -0.058104  0.010698       2        1  5.553685
1      0.015668  0.003046       3        1  5.319609
15    -0.061457  0.034542       4        1  5.250419
5     -0.070930 -0.040624       5        1  5.124217
14     0.140148  0.015128       6        1  4.673946
24    -0.048842  0.076371       7        1  4.449501
6      0.146828 -0.046431       8        1  4.211047
9     -0.108781 -0.063615       9        1  4.069449
28    -0.131916 -0.069763      10        1  3.937859
27    -0.007749  0.048791      11        1  3.629485
21    -0.054875  0.073846      12        1  3.541992
0     -0.013311  0.119638      13        1  3.522103
25     0.070368 -0.065064      14        1  3.426229
29    -0.011117  0.045702      15        1  3.346341
20     0.038507 -0.095052      16        1  3.028318
10    -0.029673 -0.063038      17        1  2.910822
4      0.065975  0.116516      18        1  2.840267
2     -0.099624 -0.098408      19        1  2.747148
17    -0.072427  0.094654      20        1  2.500800
23    -0.050206  0.082065      21        1  2.492959
26    -0.024223 -0.047620      22        1  2.196241
22     0.062613 -0.009778      23        1  2.057170
12    -0.034514  0.127617      24        1  2.047305
16     0.038651 -0.025229      25        1  1.758346
19     0.139022 -0.071066      26        1  1.655772
18     0.126724  0.003773      27        1  1.631590
3     -0.082439 -0.101141      28        1  1.495822
7      0.160770  0.019303      29        1  1.165964
8      0.017801 -0.042208      30        1  1.092419, topic_info=          Term         Freq        Total Category  logprob  loglift
1051      prio  1601.000000  1601.000000  Default  30.0000  30.0000
396        cyc  1328.000000  1328.000000  Default  29.0000  29.0000
1406        sx   932.000000   932.000000  Default  28.0000  28.0000
1038    prgnan  3208.000000  3208.000000  Default  27.0000  27.0000
961    ovuaion   824.000000   824.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
1475       wak    27.548987  1082.698363  Topic30  -5.2102   0.8455
645         im    30.934985  2777.795694  Topic30  -5.0943   0.0193
1047  prgnancy    30.668690  2977.153207  Topic30  -5.1030  -0.0587
154       baby    31.147724  4029.455759  Topic30  -5.0875  -0.3459
137         ay    26.807072  3493.041552  Topic30  -5.2375  -0.3531

[1533 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         1  0.045311   aack
0         2  0.064730   aack
0         3  0.012946   aack
0         4  0.038838   aack
0         5  0.012946   aack
...     ...       ...    ...
1579     23  0.008069  young
1579     25  0.008069  young
1579     26  0.004034  young
1579     27  0.024206  young
1579     29  0.008069  young

[19101 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 14, 2, 16, 6, 15, 25, 7, 10, 29, 28, 22, 1, 26, 30, 21, 11, 5, 3, 18, 24, 27, 23, 13, 17, 20, 19, 4, 8, 9])

#### Grid Search Num Topics for Pospartum Data

In [16]:
build_lda(pospar, 'lemma_tokens', n_topics = 30)

The number of topics is 30.
Log-Likelihood Score: -1989621.426
Perplexity Score: 3108.972


The top 15 words for topic 0
['normal', 'doctor', 'test', 'hcg', 'urine', 'headache', 'level', 'preeclampsia', 'symptom', 'bp', 'high', 'pregnancy', 'pressure', 'blood pressure', 'blood']


The top 15 words for topic 1
['normal', 'pelvic', 'belly', 'braxton hicks', 'hicks', 'low', 'contraction', 'bad', 'pregnancy', 'hurt', 'experience', 'braxton', 'not', 'cramp', 'pain']


The top 15 words for topic 2
['lactation', 'pump work', 'nipple shield', 'pumping', 'use', 'insurance', 'work', 'shield', 'flange', 'nipple', 'medela', 'breast', 'breast pump', 'spectra', 'pump']


The top 15 words for topic 3
['research', 'health', 'baby', 'woman', 'find', 'ob', 'book', 'experience', 'doctor', 'pregnant', 'not', 'class', 'pregnancy', 'hospital', 'birth']


The top 15 words for topic 4
['excited', 'depression', 'try', 'tell', 'find', 'thing', 'child', 'anxiety', 'husband', 'life', 'love', 'pregnancy', 'not', '

(LatentDirichletAllocation(n_components=30, n_jobs=-1, random_state=42),
 <49094x1749 sparse matrix of type '<class 'numpy.float64'>'
 	with 1856873 stored elements in Compressed Sparse Row format>,
 TfidfVectorizer(max_df=0.98, min_df=0.005, ngram_range=(1, 2),
                 stop_words={"'d", "'ll", "'m", "'re", "'s", "'ve", 'a', 'about',
                             'above', 'across', 'after', 'afterwards', 'again',
                             'against', 'ain', 'all', 'almost', 'alone', 'along',
                             'already', 'also', 'although', 'always', 'am',
                             'among', 'amongst', 'amount', 'an', 'and',
                             'another', ...}))

### LSA with Gensim

In [17]:
# https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [18]:
lsa_tvec = TfidfVectorizer(min_df = 0.001, ngram_range = (1,2), stop_words = full_stop_words)
X = lsa_tvec.fit_transform(health['lemma_tokens'])
X.shape

(30616, 6724)

In [19]:
svd_model = TruncatedSVD(n_components = 10, random_state = 42)
svd_model.fit(X)

TruncatedSVD(n_components=10, random_state=42)

In [20]:
terms = lsa_tvec.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:15]
    print()
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])



Topic 0: 
period
not
start
pain
pill
sex
control
help
birth
birth control
come
try
doctor
look
normal

Topic 1: 
period
pill
birth control
birth
control
bleed
cramp
pregnancy
pregnant
cycle
iud
blood
test
normal
spot

Topic 2: 
infection
yeast
yeast infection
discharge
uti
vagina
bv
smell
pain
sex
antibiotic
symptom
doctor
vaginal
test

Topic 3: 
hair
shave
yeast
infection
yeast infection
wax
wash
period
grow
dye
birth control
birth
pill
control
shampoo

Topic 4: 
birth control
control
birth
pill
yeast
yeast infection
infection
control pill
friend
pack
effect
work
good
new
start birth

Topic 5: 
pain
cyst
iud
birth
birth control
control
doctor
breast
ovarian
hurt
ovarian cyst
ovary
painful
ultrasound
experience

Topic 6: 
wear
bra
dress
look
breast
size
clothe
smell
buy
boob
makeup
jean
fit
skin
cup

Topic 7: 
iud
period
friend
pain
yeast
yeast infection
infection
cramp
cyst
work
copper
bad
heavy
copper iud
experience

Topic 8: 
iud
sex
bleed
smell
copper
copper iud
mirena
vagina
disc

In [21]:
pospar['subreddit'].value_counts()

BabyBumps               33280
breastfeeding           15593
postpartumdepression      221
Name: subreddit, dtype: int64

In [22]:
len(pospar) * 0.005

245.47